# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from random import sample

import re

import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.tsa.arima.model import ARIMA

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

# Import original data and preprocessed data

In [3]:
#sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
#calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

In [4]:
sales_train_val = pd.read_csv(r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\M5 Forecasting\Estimate the unit sales of Walmart retail goods\m5-forecasting-accuracy_unzip files\sales_train_validation.csv')
calendar = pd.read_csv(r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\M5 Forecasting\Estimate the unit sales of Walmart retail goods\m5-forecasting-accuracy_unzip files\calendar.csv')

**Note: This Step is for the extraction of California State and Foods category**

In [5]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'FOODS')]

In [6]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1612,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,3,0,0,1,...,0,2,0,4,1,1,0,1,1,0
1613,FOODS_1_002_CA_1_validation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,0,1,0,1,...,1,3,1,0,0,1,2,0,0,0
1614,FOODS_1_003_CA_1_validation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,3,0,2,1,1,0,1,0,1,0
1615,FOODS_1_004_CA_1_validation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1616,FOODS_1_005_CA_1_validation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,3,9,3,3,...,3,1,1,2,0,2,2,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12191,FOODS_3_823_CA_4_validation,FOODS_3_823,FOODS_3,FOODS,CA_4,CA,1,1,1,4,...,3,1,2,2,0,1,1,3,3,0
12192,FOODS_3_824_CA_4_validation,FOODS_3_824,FOODS_3,FOODS,CA_4,CA,0,0,0,1,...,0,1,1,1,0,2,0,0,0,1
12193,FOODS_3_825_CA_4_validation,FOODS_3_825,FOODS_3,FOODS,CA_4,CA,2,3,2,1,...,0,2,0,1,3,1,1,0,2,0
12194,FOODS_3_826_CA_4_validation,FOODS_3_826,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,1,2,2,1,4,5,1,7,0,4


In [7]:
INPUT_DIR_2 = r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\CRUNCH TIME (WEEKLY AGGREGATION)\California\Foods\Demand Pattern Categorization'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [8]:
len(sales_intermittent)

1301

In [9]:
len(sales_lumpy)

395

In [10]:
len(sales_erratic)

458

In [11]:
len(sales_smooth)

3594

**Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)**

In [12]:
sales_lumpy

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1616,FOODS_1_005_CA_1_validation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,3,9,3,3,...,3,1,1,2,0,2,2,1,4,1
1625,FOODS_1_015_CA_1_validation,FOODS_1_015,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,1,1,0,2,0,1,2,2,0
1637,FOODS_1_027_CA_1_validation,FOODS_1_027,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,5,4,2,3,1,0,2,1,3,3
1638,FOODS_1_028_CA_1_validation,FOODS_1_028,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,1,5,4,2,1,1,1,3,0,0
1640,FOODS_1_030_CA_1_validation,FOODS_1_030,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,2,0,0,1,1,2,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12147,FOODS_3_779_CA_4_validation,FOODS_3_779,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12155,FOODS_3_787_CA_4_validation,FOODS_3_787,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,0,2,2,1,0,1,0,0,3
12168,FOODS_3_800_CA_4_validation,FOODS_3_800,FOODS_3,FOODS,CA_4,CA,1,2,4,2,...,0,3,2,3,1,1,2,1,2,2
12183,FOODS_3_815_CA_4_validation,FOODS_3_815,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,1,0,2,0,0,0,0,0,0


# User-defined functions to calculate Metrics

In [13]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)


def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

# Format sales data and then merge with calendar data

In [14]:
sales_pattern = sales_lumpy.copy()

In [15]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

In [16]:
dataframe_weekly

,wm_yr_wk,FOODS_1_005_CA_1,FOODS_1_008_CA_2,FOODS_1_010_CA_2,FOODS_1_010_CA_4,FOODS_1_015_CA_1,FOODS_1_015_CA_2,FOODS_1_015_CA_3,FOODS_1_022_CA_4,FOODS_1_027_CA_1,...,FOODS_3_800_CA_3,FOODS_3_800_CA_4,FOODS_3_806_CA_1,FOODS_3_806_CA_3,FOODS_3_815_CA_2,FOODS_3_815_CA_4,FOODS_3_816_CA_2,FOODS_3_824_CA_3,FOODS_3_825_CA_2,FOODS_3_827_CA_4
0,11101,18.0,0.0,4.0,2.0,0.0,1.0,5.0,0.0,9.0,...,99.0,5.0,3.0,2.0,13.0,0.0,0.0,3.0,4.0,0.0
1,11102,31.0,0.0,2.0,0.0,1.0,3.0,1.0,0.0,20.0,...,42.0,3.0,4.0,0.0,3.0,0.0,0.0,1.0,3.0,0.0
2,11103,5.0,0.0,2.0,7.0,0.0,1.0,0.0,3.0,7.0,...,45.0,5.0,6.0,0.0,5.0,0.0,0.0,0.0,7.0,0.0
3,11104,10.0,0.0,3.0,2.0,1.0,3.0,1.0,4.0,10.0,...,64.0,8.0,12.0,0.0,0.0,0.0,0.0,1.0,10.0,0.0
4,11105,4.0,0.0,4.0,3.0,8.0,1.0,0.0,6.0,7.0,...,32.0,1.0,7.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,11608,12.0,0.0,0.0,2.0,0.0,0.0,3.0,1.0,13.0,...,57.0,8.0,11.0,12.0,0.0,0.0,3.0,6.0,6.0,0.0
269,11609,9.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,8.0,...,23.0,3.0,14.0,0.0,0.0,0.0,1.0,3.0,5.0,0.0
270,11610,13.0,0.0,3.0,0.0,1.0,2.0,2.0,0.0,17.0,...,46.0,8.0,23.0,4.0,2.0,0.0,2.0,8.0,6.0,0.0
271,11611,4.0,0.0,1.0,0.0,2.0,0.0,3.0,4.0,15.0,...,54.0,11.0,17.0,0.0,0.0,0.0,0.0,2.0,11.0,0.0


In [17]:
start_train_week = 11318
end_train_week = 11610
n_pred_weeks = 2
num_train_needed = 50
validation_training_ratio = 0.95
num_id_sample_for_tuning = 200

# Run ARIMA model with best combination

In [18]:
train_data = dataframe_weekly[(dataframe_weekly.wm_yr_wk >= start_train_week) & (dataframe_weekly.wm_yr_wk <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

In [19]:
len(list_pattern_py)

395

In [20]:
best_combination = (2, 1, 0)

In [21]:
start = timer()

df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    print('Currently Running Product: %s' % product)
    print('Progressing: {0} %'.format(round(list_pattern_py.index(product) / len(list_pattern_py) * 100, 2)))
    
    train_product = train_data[product]
    test_product = test_data[product]
    
    model = ARIMA(train_product, order = best_combination)
    model_fit = model.fit()
    forecast_temp = model_fit.forecast(len(test_product))
    df_result_temp = pd.DataFrame({'Product': [product for product_count in range(len(test_product))],
                                   'Actual Data': test_product.values,
                                   'Forecast': forecast_temp.values})
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True) 
    
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running Product: FOODS_1_005_CA_1
Progressing: 0.0 %
Currently Running Product: FOODS_1_015_CA_1
Progressing: 0.25 %
Currently Running Product: FOODS_1_027_CA_1
Progressing: 0.51 %
Currently Running Product: FOODS_1_028_CA_1
Progressing: 0.76 %
Currently Running Product: FOODS_1_030_CA_1
Progressing: 1.01 %


C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base

Currently Running Product: FOODS_1_033_CA_1
Progressing: 1.27 %
Currently Running Product: FOODS_1_037_CA_1
Progressing: 1.52 %
Currently Running Product: FOODS_1_075_CA_1
Progressing: 1.77 %
Currently Running Product: FOODS_1_095_CA_1
Progressing: 2.03 %
Currently Running Product: FOODS_1_098_CA_1
Progressing: 2.28 %
Currently Running Product: FOODS_1_107_CA_1
Progressing: 2.53 %


C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base

Currently Running Product: FOODS_1_123_CA_1
Progressing: 2.78 %
Currently Running Product: FOODS_1_126_CA_1
Progressing: 3.04 %
Currently Running Product: FOODS_1_133_CA_1
Progressing: 3.29 %
Currently Running Product: FOODS_1_151_CA_1
Progressing: 3.54 %
Currently Running Product: FOODS_1_169_CA_1
Progressing: 3.8 %
Currently Running Product: FOODS_1_176_CA_1
Progressing: 4.05 %
Currently Running Product: FOODS_1_189_CA_1
Progressing: 4.3 %
Currently Running Product: FOODS_1_193_CA_1
Progressing: 4.56 %
Currently Running Product: FOODS_1_198_CA_1
Progressing: 4.81 %
Currently Running Product: FOODS_1_204_CA_1
Progressing: 5.06 %


...

C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base

Currently Running Product: FOODS_3_663_CA_4
Progressing: 96.2 %
Currently Running Product: FOODS_3_664_CA_4
Progressing: 96.46 %
Currently Running Product: FOODS_3_707_CA_4
Progressing: 96.71 %
Currently Running Product: FOODS_3_719_CA_4
Progressing: 96.96 %
Currently Running Product: FOODS_3_733_CA_4
Progressing: 97.22 %
Currently Running Product: FOODS_3_736_CA_4
Progressing: 97.47 %
Currently Running Product: FOODS_3_749_CA_4
Progressing: 97.72 %


C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base

Currently Running Product: FOODS_3_758_CA_4
Progressing: 97.97 %
Currently Running Product: FOODS_3_763_CA_4
Progressing: 98.23 %
Currently Running Product: FOODS_3_772_CA_4
Progressing: 98.48 %
Currently Running Product: FOODS_3_779_CA_4
Progressing: 98.73 %
Currently Running Product: FOODS_3_787_CA_4
Progressing: 98.99 %
Currently Running Product: FOODS_3_800_CA_4
Progressing: 99.24 %
Currently Running Product: FOODS_3_815_CA_4
Progressing: 99.49 %
Currently Running Product: FOODS_3_827_CA_4
Progressing: 99.75 %
This line of code took 0.2580102733333334 minutes


C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base

# Calculate metrics for optimal combination

In [22]:
# Start calculating metrics 
start = timer()

df_arima_metrics = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.011534343333333178 minutes


In [23]:
df_arima_metrics

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_005_CA_1,3.298267,131.9307,80.6247,140.6228,6.596534e+00,6.640953e+00
1,FOODS_1_008_CA_2,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
2,FOODS_1_010_CA_2,inf,52.9126,37.9387,52.9126,5.291260e-01,6.588327e-01
3,FOODS_1_010_CA_4,1.637649,327.5299,125.3978,68.0886,1.637649e+00,1.896653e+00
4,FOODS_1_015_CA_1,0.459046,91.8091,151.4239,67.9204,9.180913e-01,1.018219e+00
...,...,...,...,...,...,...,...
390,FOODS_3_815_CA_4,inf,inf,200.0000,NaN,7.048247e-100,7.135712e-100
391,FOODS_3_816_CA_2,inf,inf,200.0000,NaN,1.920916e+00,1.936267e+00
392,FOODS_3_824_CA_3,2.275144,455.0287,145.2735,165.4539,4.550287e+00,4.716536e+00
393,FOODS_3_825_CA_2,0.455190,34.1392,34.3323,29.3410,2.731138e+00,3.329229e+00


In [24]:
df_arima_metrics.MAPE.mean()

86.52006919831223

In [25]:
df_arima_metrics.RMSE.mean()

2.8834597879801733

In [26]:
df_arima_metrics.to_csv('ARIMA_Lumpy_Forecast_Metrics.csv')